In [0]:
  # %pip install scalecast
  %pip install statsmodels


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%matplotlib inline

In [0]:
%run /Users/ebizindavyi@delhaize.be/Data_preparation/Parse_xls_files

## Load Libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import datetime as dt

## Load the inbound data

In [0]:
inbound_2022_dir = '/dbfs/mnt/dataplatform/acc/DataScience/sandbox/ebiz/Inbound_FC/data/input/Ecom-Ops-status_2022.xlsx'

In [0]:
inbound_2022_df = create_master_df(inbound_2022_dir)

In [0]:
inbound_2022_df

,Dates,Dry Fc,Dry Actuals,Fresh Fc,Fresh,Frozen Fc,Frozen,Ultrafresh Fc,Ultrafresh
0,2022-01-03,85794.442754,84224.0,60969.286307,41000.0,49273.053738,5581.0,6538.217201,6538.217201
1,2022-01-04,65323.939111,49564.0,68517.000793,64233.0,6447.045743,6110.0,7288.014352,9673.000000
2,2022-01-05,59060.496963,45890.0,59891.490569,61162.0,14752.884950,6061.0,8645.127518,9394.000000
3,2022-01-06,54098.434315,26976.0,54044.997867,55700.0,11884.161805,5042.0,8733.406013,7299.000000
4,2022-01-07,48633.525710,37044.0,48691.064872,49534.0,7322.811052,5979.0,7745.598367,8240.000000
...,...,...,...,...,...,...,...,...,...
293,2022-12-27,71459.270000,63764.0,67049.998422,68912.0,6642.104962,5615.0,9028.927578,11033.000000
294,2022-12-28,71916.320000,47680.0,62406.358606,57239.0,5883.382433,5253.0,9509.601394,10525.000000
295,2022-12-29,60346.346000,39552.0,76619.568000,63748.0,5951.031775,6037.0,9399.318000,10458.000000
296,2022-12-30,52203.888000,61119.0,65006.548000,42234.0,5759.074282,6692.0,9096.132000,7824.000000


### Create yearly time series
needed to fill in date gaps

In [0]:
calendar_2022 = pd.DataFrame({'Dates':pd.date_range(dt.datetime(2022,1,3), periods=363, freq='d')})
inbound_2022_dry = calendar_2022.merge(inbound_2022_df, on='Dates',how='left').fillna(0).set_index('Dates').copy()
inbound_2022_dry = inbound_2022_dry.groupby(['Dates'])['Dry Actuals'].sum()
inbound_2022_dry

Out[19]: Dates
2022-01-03    84224.0
2022-01-04    49564.0
2022-01-05    45890.0
2022-01-06    26976.0
2022-01-07    37044.0
               ...   
2022-12-27    63764.0
2022-12-28    47680.0
2022-12-29    39552.0
2022-12-30    61119.0
2022-12-31    46818.0
Name: Dry Actuals, Length: 363, dtype: float64

## Create forecasting element for ARIMA

In [0]:
training_days=90
initial_test_date= '2022-04-01'
fc_horizon_end = 21
fc_freq= 7

all_start_dates= seq(initial_test_date, fc_freq)
backtest_results=pd.DataFrame()

for start_test_date in all_start_dates:
  ## DO ARIMA
  # end_test_date =start_test_date + fc_horizon

  # start_train_date= start_test_date-training_days-1
  # end_train_date=start_train_date+training_days

  # endog=inbound_2022_dry[start_train_date:end_train_date]
  # ARIMA
  # backtest_results = backtest_results.append(forecast_df)
  pass

end_test_date =start_test_date + fc_horizon_end

start_train_date= start_test_date-training_days-1
end_train_date=start_train_date+training_days

endog=inbound_2022_dry[start_train_date:end_train_date]

In [0]:
endog = inbound_2022_dry
# exog=inbound_2022_dry.index
ar_mod = ARIMA(
    endog, order=(1, 0, 0), seasonal_order=(2, 0, 1, 7), trend=None, freq=None
)

/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/databricks/python/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


In [0]:
mod_results=ar_mod.fit()

In [0]:
mod_results.param_ter

Out[41]: ['exog', 'ar', 'seasonal_ar', 'seasonal_ma', 'variance']

In [0]:
#predict, hints: 1)specify the train period with start and end date

mod_pred=mod_results.predict(start="2023-01-01", end="2023-01-21")


In [0]:
mod_pred

Out[59]: 2023-01-01     3934.096733
2023-01-02    98475.998823
2023-01-03    70822.657009
2023-01-04    63338.321875
2023-01-05    56614.714405
2023-01-06    53043.176692
2023-01-07    52625.562916
2023-01-08     3674.924510
2023-01-09    97869.180267
2023-01-10    70401.607468
2023-01-11    62622.065725
2023-01-12    55889.663865
2023-01-13    53374.267592
2023-01-14    52390.782582
2023-01-15     4045.155175
2023-01-16    97567.828734
2023-01-17    70292.634809
2023-01-18    62581.832797
2023-01-19    55898.809902
2023-01-20    53358.449854
2023-01-21    52405.313759
Freq: D, Name: predicted_mean, dtype: float64